# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Set the path to citis.csv
cities_path = "../output_data/cities.csv"

# Read the cities.csv
cities_df = pd.read_csv(cities_path)

# # Set the index to the City ID
cities_df = cities_df.set_index('City ID')

# output cities_df
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
0,Binabalian Ricor,100,PH,1603332703,83,16.42,119.93,82.98,26.60
1,Arrondissement de Limoges,62,FR,1603332704,93,45.75,1.33,60.01,3.36
2,New Norfolk,40,AU,1603332704,44,-42.78,147.06,64.99,3.00
3,Punta Arenas,75,CL,1603332598,61,-53.15,-70.92,48.20,6.93
4,Cape Town,0,ZA,1603332499,87,-33.93,18.42,59.00,14.99
...,...,...,...,...,...,...,...,...,...
520,Bhādāsar,0,IN,1603333351,37,28.28,74.32,75.45,6.20
521,Shurugwi,100,ZW,1603333351,88,-19.67,30.01,60.69,11.90
522,Valparaíso,40,CL,1603333261,100,-33.04,-71.63,50.00,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Define weights
weights = cities_df['Humidity'].astype(float)

# Genearte figure
cities_fig = gmaps.figure()

# Define heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# Add heat layer to figure
cities_fig.add_layer(heat_layer)

# Show the heatmap
cities_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Drop any null values
cities_df = cities_df.dropna()

# Setup variables for searching for the perfect vacation spot
low_temp = 70
high_temp = 80
max_wind_speed = 10
cloudiness = 0
humidity = 70

# Create a DataFrame with only cities that have our ideal weather
vacation_cities_df = cities_df.loc[cities_df['Wind Speed'] < max_wind_speed]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Cloudiness'] ==  cloudiness]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] > low_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] < high_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Humidity'] < humidity]
vacation_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
228,Aswān,0,EG,1603332965,25,24.09,32.91,75.52,7.78
246,Bayjī,0,IQ,1603333027,28,34.93,43.49,70.12,8.10
273,Kontagora,0,NG,1603333030,61,10.40,5.47,73.99,0.43
277,Makkah al Mukarramah,0,SA,1603333030,25,21.43,39.83,72.28,1.66
309,Agadez,0,NE,1603333092,19,19.75,10.25,73.29,4.56
453,Ţahţā,0,EG,1603333285,43,26.77,31.50,73.40,6.93
520,Bhādāsar,0,IN,1603333351,37,28.28,74.32,75.45,6.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create a new dataframe and add a Hotel Name column
hotel_df = pd.DataFrame(vacation_cities_df,
                      columns=['City','Country','Lat','Lng','Hotel Name'])

hotel_df

,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
228,Aswān,EG,24.09,32.91,NaN
246,Bayjī,IQ,34.93,43.49,NaN
273,Kontagora,NG,10.40,5.47,NaN
277,Makkah al Mukarramah,SA,21.43,39.83,NaN
309,Agadez,NE,19.75,10.25,NaN
453,Ţahţā,EG,26.77,31.50,NaN
520,Bhādāsar,IN,28.28,74.32,NaN


In [6]:
# find the closest hotel for all of our possible vacation cities

# Define the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through possible vacation cities
for index, row in hotel_df.iterrows():
    
    # Set a variable for the latitude and Longitude of our city
    city_lat_lng = f"{row['Lat']},{row['Lng']}"
    
    # Define the api call parameters
    params = {
               "location": city_lat_lng,
               "radius": 5000,
               "types": "lodging",
               "key": g_key
             }

    # Output that we are retrieving information for our city
    print(f"Retrieving Results for Index {index}: {row['City']}, {row['Country']}.")
   
    # assemble url and make API request
    response = requests.get(base_url, params=params)
    
    print(response)
    response_json = response.json()

    # extract results
    results = response_json['results']

    # If we got a hotel add it to the DataFrame
    # if we don't get a hotel then output a message that we are skipping it
    try:
        # Output the closet hotel
        print(f"Closest hotel is {results[0]['name']}.")
        
        # Update the data frame with the hotel name
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
 
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-----------------------------------------------------")

# Output hotel_df
hotel_df   

Retrieving Results for Index 228: Aswān, EG.
<Response [200]>
Closest hotel is Nuba Nile.
-----------------------------------------------------
Retrieving Results for Index 246: Bayjī, IQ.
<Response [200]>
Closest hotel is قريه جديده.
-----------------------------------------------------
Retrieving Results for Index 273: Kontagora, NG.
<Response [200]>
Closest hotel is City Gate Inn.
-----------------------------------------------------
Retrieving Results for Index 277: Makkah al Mukarramah, SA.
<Response [200]>
Closest hotel is Hotel Fairmont Makkah Clock Royal Tower.
-----------------------------------------------------
Retrieving Results for Index 309: Agadez, NE.
<Response [200]>
Missing field/result... skipping.
-----------------------------------------------------
Retrieving Results for Index 453: Ţahţā, EG.
<Response [200]>
Closest hotel is Møhãmęd Tãręk 🏠.
-----------------------------------------------------
Retrieving Results for Index 520: Bhādāsar, IN.
<Response [200]>
Clos

,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
228,Aswān,EG,24.09,32.91,Nuba Nile
246,Bayjī,IQ,34.93,43.49,قريه جديده
273,Kontagora,NG,10.40,5.47,City Gate Inn
277,Makkah al Mukarramah,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
309,Agadez,NE,19.75,10.25,NaN
453,Ţahţā,EG,26.77,31.50,Møhãmęd Tãręk 🏠
520,Bhādāsar,IN,28.28,74.32,OMPRAKASH JAKHAR


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
cities_fig.add_layer(markers)

# Display Map
cities_fig

Figure(layout=FigureLayout(height='420px'))